# Embedding Matplotlib Animations in Jupyter as Interactive JavaScript Widgets

Written by Jean M Favre, Swiss National Supercomputing Center.
Tested on Piz Daint Thu 01 Oct 2020 08:38:49 AM CEST

See http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-as-interactive-javascript-widgets/

On Piz Daint, a video can also be saved to disk using the FFmpeg module.

put the following line in your $HOME/.jupyterhub.env

_module load FFmpeg/4.2.2-CrayGNU-20.08_

using the _inline_ backend, the output of plotting commands is displayed directly below the code cell that produced it.

The resulting plots will then also be stored in the notebook document.
The image resolution defaults to 432x288 pixels, i.e. a figsize of 6x4, at 72 dpi

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib
matplotlib.use("Agg")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
#plt.rcParams['figure.figsize'] = [9.5, 6]

In [ ]:
fig, ax = plt.subplots()

ax.set_xlim(( 0, 2))
ax.set_ylim((-2, 2))

line, = ax.plot([], [], lw=2)

In [ ]:
def init():
    line.set_data([], [])
    return (line,)

In [ ]:
def animate(i):
    x = np.linspace(0, 2, 1000)
    y = np.sin(2 * np.pi * (x - 0.01 * i))
    line.set_data(x, y)
    return (line,)

In [ ]:
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, 
                               blit=True)

In [ ]:
writervideo = animation.FFMpegWriter(fps=60, codec='mpeg4') 

anim.save("/users/jfavre/movie2.mp4", writer=writervideo)

By default, we don't get any sort of animation from the inline display

In [ ]:
anim._repr_html_() is None

Change the default, to display the animation as HTML with JavaScript

In [ ]:
rc('animation', html='jshtml')
anim